ANN Doodle Classfier

In [ ]:
""" FCNN for Doodle Classification. """

##### Libraries

import torch
import torch.nn as nn
import torch.nn.functional as F

# =============================================================================
# Network Architecture
# =============================================================================

class DoodleANN(nn.Module):

    def __init__(self):
        # Init
        super(DoodleANN, self).__init__()

        # Neurons
        self.input_layer = 28*28 # features from flattened 28*28 image
        self.hidden_layer_1 = 300 # neurons
        self.hidden_layer_2 = 60 # neurons
        self.hidden_layer_3 = 50 # neurons
        self.hidden_layer_4 = 40 # neurons
        self.output_layer = 10 # categories

        # Architecture
        self.fc1 = nn.Linear(self.input_layer, self.hidden_layer_1)
        self.fc2 = nn.Linear(self.hidden_layer_1, self.hidden_layer_2)
        self.fc3 = nn.Linear(self.hidden_layer_2, self.hidden_layer_3)
        self.fc4 = nn.Linear(self.hidden_layer_3, self.hidden_layer_4)
        self.output = nn.Linear(self.hidden_layer_4, self.output_layer)

    def forward(self, x):
        # Flattening the input tensor
        x = x.view(-1, self.input_layer)

        x = F.relu(self.fc1(x))     # Relu activation function for hidden layer 1
        x = F.relu(self.fc2(x))     # Relu activation function for hidden layer 2
        x = F.relu(self.fc3(x))     # Relu activation function for hidden layer 3
        x = F.relu(self.fc4(x))     # Relu activation function for hidden layer 4
        x = self.output(x)

        return x

Train DoodleANN

In [ ]:
""" Train network here. Persistence to store weights. """

##### Libraries

import numpy as np
import os
import pandas as pd
import torch.optim as optim
import torch.nn as nn
import torch
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

# =============================================================================
# Directory Path
# =============================================================================

model_save_path = '/content/drive/My Drive/Your_Folder/doodle_ann_weights.pth'

# =============================================================================
# Train Network
# =============================================================================

## Drive
drive.mount('/content/drive/')

## Get data ready
print("Getting data ready to train...")
df = pd.read_csv('/content/drive/My Drive/Doodle/doodle_dataframe.csv')
pixel_columns = [f'pixel{i}' for i in range(28*28)]
X = df[pixel_columns].values
y = df['label'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Label Encoder to change string categorical labels to numbers
encoder = joblib.load('/content/doodle_category_encoder.joblib')
y_train_encoded = encoder.transform(y_train)
y_test_encoded = encoder.transform(y_test)

## Train model
print("Training...")
model = DoodleANN()
# print("Loading weights...")
# model.load_state_dict(torch.load('/content/doodle_ann_weights.pth'))
model.train()  # set model to training mode

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Prepare data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)      # ensure long type for CrossEntropyLoss

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

# Training Loop
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch}], Loss: {loss.item():.4f}')

    if epoch % 10 == 0:
      # Save weights
      print("Persistence: Saving model weights...")
      torch.save(model.state_dict(), '/content/doodle_ann_weights_100_4layrs.pth')

# Save weights
print("Finished training. Saving model weights!")
torch.save(model.state_dict(), '/content/doodle_ann_weights_100_4layers.pth')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Getting data ready to train...
Training...
Epoch [0], Loss: 2.3063
Persistence: Saving model weights...
Epoch [1], Loss: 2.3024
Epoch [2], Loss: 2.2983
Epoch [3], Loss: 2.2931
Epoch [4], Loss: 2.2871
Epoch [5], Loss: 2.2798
Epoch [6], Loss: 2.2714
Epoch [7], Loss: 2.2611
Epoch [8], Loss: 2.2484
Epoch [9], Loss: 2.2336
Epoch [10], Loss: 2.2162
Persistence: Saving model weights...
Epoch [11], Loss: 2.1961
Epoch [12], Loss: 2.1731
Epoch [13], Loss: 2.1479
Epoch [14], Loss: 2.1206
Epoch [15], Loss: 2.0915
Epoch [16], Loss: 2.0617
Epoch [17], Loss: 2.0314
Epoch [18], Loss: 2.0010
Epoch [19], Loss: 1.9714
Epoch [20], Loss: 1.9434
Persistence: Saving model weights...
Epoch [21], Loss: 1.9174
Epoch [22], Loss: 1.8938
Epoch [23], Loss: 1.8728
Epoch [24], Loss: 1.8552
Epoch [25], Loss: 1.8403
Epoch [26], Loss: 1.8264
Epoch [27], Loss: 1.8142
Epoch [28], Loss: 1.8028


In [ ]:
# # Use 'inverse_transform' to convert numerical predictions back to original labels
# predicted_labels = model_output.numpy()  # Assuming model_output is a tensor
# predicted_categories = encoder.inverse_transform(predicted_labels)

Accuracy of Model

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

def evaluate_accuracy(data_loader, model):
    correct = 0
    total = 0
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No gradients needed for evaluation, which saves memory and computations
        for data in data_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

train_accuracy = evaluate_accuracy(train_loader, model)
test_accuracy = evaluate_accuracy(test_loader, model)

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

Training Accuracy: 81.41%
Test Accuracy: 79.90%


In [35]:
# Prevent Colab from timing out
while True: pass

KeyboardInterrupt: 